In [ ]:
InstructionStr = str
BitFilter = str

OPTABLE_FILTERS : dict[BitFilter, InstructionStr] = {
    # Block 0
    '0000_0000': 'instr_nop',
    '00xx_0001': 'instr_ld_r16_imm16',
    '00xx_0010': 'instr_ld_r16mem_a',
    '00xx_1010': 'instr_ld_a_r16mem',
    '0000_1000': 'instr_imm16mem_sp',
    
    '00xx_0011': 'instr_inc_r16',
    '00xx_1011': 'instr_dec_r16',
    '00xx_1001': 'instr_add_hl_r16',
    '00xx_x100': 'instr_inc_r8',
    '00xx_x101': 'instr_dec_r8',  # decr
    '00xx_x110': 'instr_ld_r8_imm8', # ld r8 imm8
    
    '0000_0111': 'instr_rlca', 
    '0000_1111': 'instr_rrca', 
    '0001_0111': 'instr_rla', 
    '0001_1111': 'instr_rra',
    '0010_0111': 'instr_daa',
    '0010_1111': 'instr_cpl',
    '0011_0111': 'instr_scf',
    '0011_1111': 'instr_ccf',
    
    '0001_1000': 'instr_jr_imm8',
    '001x_x000': 'instr_jr_cond_imm8',
    '0001_0000': 'instr_stop',

    # Block 1
    '01xx_xyyy': 'instr_ld_r8_r8',
    '0111_0110': 'instr_halt',
    
    # Block 2 (8 bit ALU operations)
    '10xx_xyyy': 'instr_alu_op_r8',
    
    # Block 3
    '11xx_x110': 'instr_alu_op_imm8',    # immediate 8-bit alu
    '110x_x000': 'instr_ret_cond',
    '1100_1001': 'instr_ret',
    '1101_1001': 'instr_reti',
    '110x_x010': 'instr_jp_cond',
    '1100_0011': 'instr_jp_imm16',
    '1110_1001': 'instr_jp_hl',
    '110x_x100': 'instr_call_cond_imm16',
    '1100_1101': 'instr_call_imm16',
    '11xx_x111': 'instr_rst_tgt3',
    
    '11xx_0001': 'instr_pop_r16stk',
    '11xx_0101': 'instr_push_r16stk',
    
    '1100_1011': 'instr_cb_prefix',
    
    '1110_0010': 'instr_ldh_cmem_a',
    '1110_0000': 'instr_ldh_imm8mem_a',
    '1110_1010': 'instr_ld_imm16mem_a',
    '1111_0010': 'instr_ldh_a_cmem',
    '1111_1010': 'instr_ld_a_imm16mem',
    
    '1110_1000': 'instr_add_sp_imm8',
    '1111_1000': 'instr_ld_hl_sp_imm8',
    '1111_1001': 'instr_ld_sp_hl',
    
    '1111_0011': 'instr_di',
    '1111_1011': 'instr_ei',
}

In [19]:
from collections import Counter

def generate_expansion(og_str: str, index: int, generated_str, str_set: set):
    if (index == len(og_str)):
        str_set.add(generated_str)
        return
    
    s_i = og_str[index]
    if s_i != '0' and s_i != '1':
        generate_expansion(og_str, index+1, generated_str + '0', str_set)
        generate_expansion(og_str, index+1, generated_str + '1', str_set)
    else:
        generate_expansion(og_str, index+1, generated_str + s_i, str_set)

def expand_bitfilter(bitfilter: BitFilter):
    bitfilter = bitfilter.replace('_', '')
    replacement_sets = set()
    generate_expansion(bitfilter, 0, '', replacement_sets)

    return replacement_sets


expand_bitfilter('1111')            
    

{'1111'}

In [21]:
def create_optable():
    optable = {}
    
    for bitfilter, func_name in OPTABLE_FILTERS.items():
        expansion_set = expand_bitfilter(bitfilter)
        for op_byte in expansion_set:
            optable[int(op_byte, 8)] = func_name
            
    return optable

In [24]:
optable = create_optable()
sorted_optable = dict(sorted(optable.items()))
print(sorted_optable)
print(len(sorted_optable))

{0: 'instr_nop', 1: 'instr_ld_r16_imm16', 8: 'instr_ld_r16mem_a', 9: 'instr_inc_r16', 64: 'instr_inc_r8', 65: 'instr_dec_r8', 72: 'instr_ld_r8_imm8', 73: 'instr_rlca', 512: 'instr_imm16mem_sp', 513: 'instr_add_hl_r16', 520: 'instr_ld_a_r16mem', 521: 'instr_dec_r16', 576: 'instr_inc_r8', 577: 'instr_dec_r8', 584: 'instr_ld_r8_imm8', 585: 'instr_rrca', 4096: 'instr_stop', 4097: 'instr_ld_r16_imm16', 4104: 'instr_ld_r16mem_a', 4105: 'instr_inc_r16', 4160: 'instr_inc_r8', 4161: 'instr_dec_r8', 4168: 'instr_ld_r8_imm8', 4169: 'instr_rla', 4608: 'instr_jr_imm8', 4609: 'instr_add_hl_r16', 4616: 'instr_ld_a_r16mem', 4617: 'instr_dec_r16', 4672: 'instr_inc_r8', 4673: 'instr_dec_r8', 4680: 'instr_ld_r8_imm8', 4681: 'instr_rra', 32768: 'instr_jr_cond_imm8', 32769: 'instr_ld_r16_imm16', 32776: 'instr_ld_r16mem_a', 32777: 'instr_inc_r16', 32832: 'instr_inc_r8', 32833: 'instr_dec_r8', 32840: 'instr_ld_r8_imm8', 32841: 'instr_daa', 33280: 'instr_jr_cond_imm8', 33281: 'instr_add_hl_r16', 33288: 'instr

In [17]:
n = 42
bin(n)

'0b101010'